# hr 数据预处理


In [38]:
import re

from collections import Counter
from my_py_toolkit.file.file_toolkit import read_file, readjson, writejson, get_file_paths

In [39]:
txt = '1、[@[@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Product*]#Relation_ide*]'
reg_rel = '\[@(?P<rel>.+)\#Relation_ide\*\]'

In [40]:
match = re.search(reg_rel, txt)
match.groupdict(), match.group()

({'rel': '[@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Product*]'},
 '[@[@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Product*]#Relation_ide*]')

In [41]:
txt = '[@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Product*]222'
reg_ner = '\[@(?P<ner_content>.*?)\#(?P<ner_name>.*?)\*\]'
for match in re.finditer(reg_ner, txt):
    print(match.groupdict())

{'ner_content': '投资计划名称', 'ner_name': 'Common_att'}
{'ner_content': '百年一武汉襄阳大厦不动产债权投资计划', 'ner_name': 'Product'}


In [57]:
def handle_ner(txt):
    reg_ner = '\[[@$](?P<ner_content>.*?)\#(?P<ner_name>.*?)\*\]'
    pre_end = 0
    txt_handle = ''
    labels = []
    for match in re.finditer(reg_ner, txt):
        s, e = match.span()
        txt_handle += txt[pre_end:s]
        pre_end = e
        ner_name = match['ner_name']
        ner_content = match['ner_content']
        labels.append((ner_name, ner_content, (len(txt_handle), len(txt_handle) + len(ner_content))))
        txt_handle += ner_content
    txt_handle += txt[pre_end:]
    return txt_handle, labels



In [43]:
handle_ner(txt)

('投资计划名称：百年一武汉襄阳大厦不动产债权投资计划222',
 [('Common_att', '投资计划名称', (0, 6)),
  ('Product', '百年一武汉襄阳大厦不动产债权投资计划', (7, 25))])

In [44]:
match.span(), match['ner_name'], match.group('ner_name')

((22, 52), 'Product', 'Product')

In [58]:
def handle(txt):
    reg_rel = '\[[@$](?P<rel>.+)\#Relation_ide\*\]'
    pre_end = 0
    txt_handle = ''
    labels = []
    for match in re.finditer(reg_rel, txt):
        s, e = match.span()
        txt_rel = match['rel']
        # print(f'rel: {txt_rel}')
        txt_handle += txt[pre_end:s]
        pre_end = e
        txt_ner, label_sub = handle_ner(txt_rel)
        for i, item in enumerate(label_sub):
            label_sub[i] = (item[0], item[1], [v + len(txt_handle) for v in item[2]])
           
        labels.append(label_sub)
        txt_handle += txt_ner
    
    if pre_end < len(txt):
        txt_ner, label_sub = handle_ner(txt[pre_end:])
        for item in label_sub:
            scope = [v + len(txt_handle) for v in item[2]]
            labels.append([(item[0], item[1], scope)])
        txt_handle += txt_ner
    
    return txt_handle, labels
        


In [46]:
txt = '1、[@[@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Product*]#Relation_ide*]'

In [47]:
handle(txt)

('1、投资计划名称：百年一武汉襄阳大厦不动产债权投资计划',
 [[('Common_att', '投资计划名称', [2, 8]),
   ('Product', '百年一武汉襄阳大厦不动产债权投资计划', [9, 27])]])

In [48]:
p = '../resources/dataset/hr/61_6.1 百年-武汉襄阳大厦不动产债权投资计划-募集说明书.pdf.txt.ann'
datas = read_file(p, '\n')
len(datas)

FileNotFoundError: [Errno 2] No such file or directory: '../resources/dataset/hr/61_6.1 百年-武汉襄阳大厦不动产债权投资计划-募集说明书.pdf.txt.ann'

In [50]:
# 处理所有数据
paths = get_file_paths('../resources/dataset/hr/', ['ann'])
len(paths)

351

In [51]:
datas = []
for p in paths:
    datas.extend(read_file(p, '\n'))
len(datas)

220786

In [64]:
datas_handled = []

for line in datas:
    txt, labels = handle(line)
    if labels:
        datas_handled.append((txt, labels))
len(datas_handled)

85714

In [65]:
writejson(datas_handled, '../cache/hr_data.json')

In [60]:
datas_handled[:3]

[('合众资产合众-山东水发大厦不动产债权投资计划募集说明书',
  [[('document', '合众资产合众-山东水发大厦不动产债权投资计划募集说明书', [0, 27])]]),
 ('为保护委托人及受益人的合法权益，明确投资计划当事人的权利和义务，规范投资计划运作，受托人将在募集范围内公布本募集说明书，并与委托人分别签署《合众-山东水发大厦不动产债权投资计划受托合同》（以下简称《受托合同》）。本募集说明书与《受托合同》内容不一致之处，以《受托合同》为准。',
  [[('document', '《合众-山东水发大厦不动产债权投资计划受托合同》', [69, 93])],
   [('document', '《受托合同》', [98, 104])],
   [('document', '《受托合同》', [113, 119])],
   [('document', '《受托合同》', [128, 134])]]),
 ('本募集说明书在向中国保监会、委托人及受益人说明投资计划的募集资金用途、交易结构、信用增级方式、各方当事人基本情况、募集方式、受益凭证要素与转让安排、投资计划收益及分配、风险揭示与说明及管理对策、信息披露等具体内容。',
  [[('Organization', '中国保监会', [8, 13])]])]

In [62]:
key = '主要原因：一是合并范围变化，减少了金丹公司一家子公司，其他应收款减少；二是收回往'
for l in datas:
    if key in l:
        print(l)

公司其他应收款大部分为与[$广西高速公路还贷运营管理中心#Organization*]的往来款，[@2018年末占比为43.89%#Number*]。[$2016年末#Date*]、[$2017年末#Date*]、[$2018年末#Date*]和[$2019年6月末#Date*]，公司的其他应收款分别为[@77.36亿元#Number_alpha*]、[@66.53亿元#Number_alpha*]、[@187.47亿元#Number_alpha*]和[@205.86亿元#Number_alpha*]。[$2017年末#Date*]较[$2016年末#Date*]减少[@10.83亿元#Number_alpha*]，降幅为[@14.00%#Number_alpha*，主要原因：一是合并范围变化，减少了金丹公司一家子公司，其他应收款减少；二是收回往来款增加；三是计提了应收账款减值准备其他应收款账准备有所增加。2018年末其他应收款较年初增长[@181.78%#Number_alpha*]，主要原因为公司与[$广西铁路投资集团有限公司#Organization*]实施战略重组，财务数据合并所致。


In [63]:
txt = '公司其他应收款大部分为与[$广西高速公路还贷运营管理中心#Organization*]的往来款，[@2018年末占比为43.89%#Number*]。[$2016年末#Date*]、[$2017年末#Date*]、[$2018年末#Date*]和[$2019年6月末#Date*]，公司的其他应收款分别为[@77.36亿元#Number_alpha*]、[@66.53亿元#Number_alpha*]、[@187.47亿元#Number_alpha*]和[@205.86亿元#Number_alpha*]。[$2017年末#Date*]较[$2016年末#Date*]减少[@10.83亿元#Number_alpha*]，降幅为[@14.00%#Number_alpha*，主要原因：一是合并范围变化，减少了金丹公司一家子公司，其他应收款减少；二是收回往来款增加；三是计提了应收账款减值准备其他应收款账准备有所增加。2018年末其他应收款较年初增长[@181.78%#Number_alpha*]，主要原因为公司与[$广西铁路投资集团有限公司#Organization*]实施战略重组，财务数据合并所致。'
txt, labels = handle(txt)
txt, labels

('公司其他应收款大部分为与广西高速公路还贷运营管理中心的往来款，2018年末占比为43.89%。2016年末、2017年末、2018年末和2019年6月末，公司的其他应收款分别为77.36亿元、66.53亿元、187.47亿元和205.86亿元。2017年末较2016年末减少10.83亿元，降幅为14.00%，主要原因为公司与广西铁路投资集团有限公司实施战略重组，财务数据合并所致。',
 [[('Organization', '广西高速公路还贷运营管理中心', [12, 26])],
  [('Number', '2018年末占比为43.89%', [31, 46])],
  [('Date', '2016年末', [47, 53])],
  [('Date', '2017年末', [54, 60])],
  [('Date', '2018年末', [61, 67])],
  [('Date', '2019年6月末', [68, 76])],
  [('Number_alpha', '77.36亿元', [88, 95])],
  [('Number_alpha', '66.53亿元', [96, 103])],
  [('Number_alpha', '187.47亿元', [104, 112])],
  [('Number_alpha', '205.86亿元', [113, 121])],
  [('Date', '2017年末', [122, 128])],
  [('Date', '2016年末', [129, 135])],
  [('Number_alpha', '10.83亿元', [137, 144])],
  [('Number_alpha*，主要原因：一是合并范围变化，减少了金丹公司一家子公司，其他应收款减少；二是收回往来款增加；三是计提了应收账款减值准备其他应收款账准备有所增加。2018年末其他应收款较年初增长[@181.78%#Number_alpha',
    '14.00%',
    [148, 154])],
  [('Organization', '广西铁路投资集团有限公司', [163, 175])]])

In [67]:
all_ner = []
for d in datas_handled:
    for labels in d[1]:
        all_ner.extend([v[0] for v in labels])
all_ner = set(all_ner)
all_ner

{'#Number_alpha',
 'Comm[@on_att',
 'Common_att',
 'Common_att*[@]0.77亿元#Number_alpha',
 'Common_att*[@]1,680.45亿元#Number_alpha',
 'Common_att*[@]2140万元#Number_alpha',
 'Common_att*[@]9.79亿元#Number_alpha',
 'Common_att*[@]昆明保利房地产开发有限公司#Organization',
 'Dae',
 'Date',
 'Date* ]至今任[@山西焦煤集团有限责任公司#Organization',
 'Date*，[@[@总资产#Common_att',
 'Dat增加[@53.53亿元#Number',
 'Level',
 'Location',
 'Nmber_alpha',
 'Numbe_alpha',
 'Number',
 'Number*#Relation_ide',
 'Number_alha',
 'Number_alpha',
 'Number_alpha*#Relation_ide',
 'Number_alpha*[@1年#Period',
 'Number_alpha*[@]政府性债务#Common_att',
 'Number_alpha*，[@[@加权平均贷款利率#Common_att',
 'Number_alpha*，主要原因：一是合并范围变化，减少了金丹公司一家子公司，其他应收款减少；二是收回往来款增加；三是计提了应收账款减值准备其他应收款账准备有所增加。2018年末其他应收款较年初增长[@181.78%#Number_alpha',
 'Number_att',
 'Orgaization',
 'Organization',
 'Organization* 指定机构出具投资计划注册文件之日起[@【三】个月内#Period',
 'Organization*[@]2017年#Date',
 'Organization*[[@2015年#Date',
 'Organization*（以下简称“[@中国平安集团#Organization',
 'Organization_att',
 'Perid',
 'Perio

In [1]:
labels = ['Number_alpha', 'Common_att', 'Date', 'Level', 'Location', 'Number', 'Period', 'Organization', 'Product', 'Number_att', 'Organization_att', 'Person', 'Person_att', 'dcument', ]
len(labels), len(set(labels))

(14, 14)

In [37]:
# 统计实体数量，每种关系内实体的数量统计
ner_count = 0
ner_count_per_rel = []
for d in datas_handled:
    ner_count += sum([len(v) for v in d[1]])
    ner_count_per_rel.extend([len(v) for v in d[1]])

ner_count, Counter(ner_count_per_rel)

(266790,
 Counter({1: 132794,
          2: 13159,
          6: 1166,
          4: 2065,
          8: 748,
          7: 762,
          5: 1025,
          11: 526,
          9: 542,
          12: 454,
          3: 1007,
          10: 569,
          0: 779,
          15: 274,
          14: 291,
          13: 387,
          22: 96,
          18: 174,
          20: 102,
          16: 251,
          23: 75,
          32: 30,
          52: 1,
          19: 120,
          37: 16,
          26: 40,
          17: 189,
          21: 89,
          25: 61,
          27: 51,
          30: 39,
          35: 13,
          24: 67,
          38: 13,
          44: 6,
          28: 33,
          36: 17,
          63: 2,
          42: 8,
          41: 8,
          60: 2,
          48: 3,
          31: 27,
          29: 27,
          40: 10,
          39: 14,
          34: 18,
          70: 1,
          33: 16,
          43: 4,
          50: 3,
          82: 1,
          46: 4,
          73: 1,
          54

In [26]:
datas_handle[:2]

[('百年一武汉襄阳大厦不动产债权投资计划募集说明书',
  [[('document', '百年一武汉襄阳大厦不动产债权投资计划募集说明书', [0, 23])]]),
 ('百年一武汉襄阳大厦不动产债权投资计划募集说明书',
  [[('document', '百年一武汉襄阳大厦不动产债权投资计划募集说明书', [0, 23])]])]

In [18]:
res = handle(txt)
res

rel: [@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Product*]


('1、投资计划名称：百年一武汉襄阳大厦不动产债权投资计划[@投资计划名称：百年一武汉襄阳大厦不动产债权投资计划#Relation_ide*]',
 [[('Common_att', '投资计划名称', [2, 8]),
   ('Product', '百年一武汉襄阳大厦不动产债权投资计划', [9, 27])],
  [('Common_att', '[@投资计划名称', [27, 35])],
  [('Product', '百年一武汉襄阳大厦不动产债权投资计划', [36, 54])]])

In [19]:
len(res[1])

3

In [20]:
res[1]

[[('Common_att', '投资计划名称', [2, 8]),
  ('Product', '百年一武汉襄阳大厦不动产债权投资计划', [9, 27])],
 [('Common_att', '[@投资计划名称', [27, 35])],
 [('Product', '百年一武汉襄阳大厦不动产债权投资计划', [36, 54])]]

In [8]:
reg_rel = '\[@(?P<rel>.+)\#Relation_ide\*\]'
re.search(reg_rel, txt)

<_sre.SRE_Match object; span=(2, 71), match='[@[@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Prod>

In [9]:
for match in re.finditer(reg_rel, txt):
    print(match.group())

[@[@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Product*]#Relation_ide*]


In [10]:
match['rel']

'[@投资计划名称#Common_att*]：[@百年一武汉襄阳大厦不动产债权投资计划#Product*]'